<a href="https://colab.research.google.com/github/denklewer/ray-custom-agents/blob/master/trpo_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!sudo apt-get install -y build-essential curl unzip psmisc
!pip install cython==0.29.0
!git clone https://github.com/ray-project/ray.git
!ray/ci/travis/install-bazel.sh
!pip install lz4

In [0]:
!mv ray ray-distr
!pip install -e ray-distr/python/. --verbose  # Add --user if you see a permission denied error.

In [0]:
import sys
import os
sys.path.append(os.path.join("", 'ray-distr/python')) # To find local version of the library

In [21]:
!rm -r ray-custom-agents
!git clone https://github.com/denklewer/ray-custom-agents.git


Cloning into 'ray-custom-agents'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 34 (delta 9), reused 28 (delta 6), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [0]:

sys.path.append(os.path.join("", 'ray-custom-agents')) # To find local version of the library


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import trpo
import gym
import ray
from trpo import TRPOAgent, DEFAULT_CONFIG
from ray.tune.logger import pretty_print


In [24]:
ray.init(ignore_reinit_error=True)

2019-05-17 21:15:24,677	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-17 21:15:24,679	ERROR worker.py:1347 -- Calling ray.init() again after it has already been called.


In [25]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 2
config['num_sgd_iter'] = 30
config['num_gpus'] = 3
config['sgd_minibatch_size'] = 128
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = TRPOAgent(config, 'CartPole-v0')

2019-05-17 21:15:28,167	WARNING __init__.py:22 -- DeprecationWarning: TRPOAgent has been renamed to TRPOTrainer. This will raise an error in the future.
FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-05-17 21:15:28,189	INFO policy_evaluator.py:312 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-17 21:15:29,282	INFO policy_evaluator.py:732 -- Built policy map: {'default_policy': <trpo.trpo_policy_graph.TRPOPolicyGraph object at 0x7fcddc8ee390>}
2019-05-17 21:15:29,283	INFO policy_evaluator.py:733 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fcd

(pid=12032) 2019-05-17 21:15:29,652	WARNING compression.py:20 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(pid=12037) 2019-05-17 21:15:29,683	WARNING compression.py:20 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(pid=12032) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=12032) Instructions for updating:
(pid=12032) non-resource variables are not supported in the long term
(pid=12037) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pi

2019-05-17 21:15:39,491	ERROR worker.py:1616 -- Possible unhandled error from worker: ray_worker (pid=12032, host=f94ada20656f)
  File "pyarrow/serialization.pxi", line 458, in pyarrow.lib.deserialize
  File "pyarrow/serialization.pxi", line 421, in pyarrow.lib.deserialize_from
  File "pyarrow/serialization.pxi", line 272, in pyarrow.lib.SerializedPyObject.deserialize
  File "pyarrow/serialization.pxi", line 171, in pyarrow.lib.SerializationContext._deserialize_callback
ModuleNotFoundError: No module named 'trpo'



In [12]:
for i in range(2):
    result = agent.train()
    print(pretty_print(result))

(pid=500) 2019-05-17 13:19:13,514	INFO policy_evaluator.py:437 -- Generating sample batch of size 200
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:308 -- Raw obs from env: { 0: { 'agent0': np.ndarray((4,), dtype=float64, min=-0.031, max=0.005, mean=-0.013)}}
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:309 -- Info return from env: {0: {'agent0': None}}
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:407 -- Preprocessed obs: np.ndarray((4,), dtype=float64, min=-0.031, max=0.005, mean=-0.013)
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:411 -- Filtered obs: np.ndarray((4,), dtype=float64, min=-0.031, max=0.005, mean=-0.013)
(pid=500) 2019-05-17 13:19:13,516	INFO sampler.py:525 -- Inputs to compute_actions():
(pid=500) 
(pid=500) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(pid=500)                                   'env_id': 0,
(pid=500)                                   'info': None,
(pid=500)                                   'obs': np.ndarray((4,), dtype=f